In [2]:
from __future__ import print_function
import argparse
import torch
import torch.utils.data
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torchvision import datasets, transforms

In [3]:
log_interval = 10
seed = 1
epochs = 10
batch_size = 128
no_cuda = False
cuda =  True


In [11]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(784, 400)
        self.fc21 = nn.Linear(400, 20)
        self.fc22 = nn.Linear(400, 20)
        self.fc3 = nn.Linear(20, 400)
        self.fc4 = nn.Linear(400, 784)

        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def encode(self, x):
        h1 = self.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparametrize(self, mu, logvar):
        std = logvar.mul(0.5).exp_()
        if cuda:
            eps = torch.cuda.FloatTensor(std.size()).normal_()
        else:
            eps = torch.FloatTensor(std.size()).normal_()
        eps = Variable(eps)
        return eps.mul(std).add_(mu)

    def decode(self, z):
        h3 = self.relu(self.fc3(z))
        return self.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 784))
        z = self.reparametrize(mu, logvar)
        return self.decode(z), mu, logvar


In [12]:
def loss_function(recon_x, x, mu, logvar):
    BCE = reconstruction_function(recon_x, x)

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD_element = mu.pow(2).add_(logvar.exp()).mul_(-1).add_(1).add_(logvar)
    KLD = torch.sum(KLD_element).mul_(-0.5)

    return BCE + KLD


In [13]:
def test(epoch):
    model.eval()
    test_loss = 0
    for data, _ in test_loader:
        if cuda:
            data = data.cuda()
        data = Variable(data, volatile=True)
        recon_batch, mu, logvar = model(data)
        test_loss += loss_function(recon_batch, data, mu, logvar).data[0]

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [14]:
torch.manual_seed(seed)
if cuda:
    torch.cuda.manual_seed(seed)


kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True, **kwargs)


Files already downloaded


In [15]:
model = VAE()
if cuda:
    model.cuda()

reconstruction_function = nn.BCELoss()
reconstruction_function.size_average = False
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [16]:
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = Variable(data)
        if cuda:
            data = data.cuda()
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.data[0]
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.data[0] / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))

In [17]:
for epoch in range(1, epochs + 1):
    train(epoch)
    test(epoch)

Train Epoch: 1 [0/60000 (0%)]	Loss: 549.307800
Train Epoch: 1 [1280/60000 (2%)]	Loss: 311.565613
Train Epoch: 1 [2560/60000 (4%)]	Loss: 236.338989
Train Epoch: 1 [3840/60000 (6%)]	Loss: 225.371078
Train Epoch: 1 [5120/60000 (9%)]	Loss: 208.993668
Train Epoch: 1 [6400/60000 (11%)]	Loss: 206.427368
Train Epoch: 1 [7680/60000 (13%)]	Loss: 208.580597
Train Epoch: 1 [8960/60000 (15%)]	Loss: 201.324646
Train Epoch: 1 [10240/60000 (17%)]	Loss: 191.824326
Train Epoch: 1 [11520/60000 (19%)]	Loss: 195.214203
Train Epoch: 1 [12800/60000 (21%)]	Loss: 189.770447
Train Epoch: 1 [14080/60000 (23%)]	Loss: 173.119644
Train Epoch: 1 [15360/60000 (26%)]	Loss: 179.030197
Train Epoch: 1 [16640/60000 (28%)]	Loss: 170.247345
Train Epoch: 1 [17920/60000 (30%)]	Loss: 169.193451
Train Epoch: 1 [19200/60000 (32%)]	Loss: 162.828690
Train Epoch: 1 [20480/60000 (34%)]	Loss: 158.171326
Train Epoch: 1 [21760/60000 (36%)]	Loss: 158.530518
Train Epoch: 1 [23040/60000 (38%)]	Loss: 155.896255
Train Epoch: 1 [24320/60000 